In [2]:
import gzip
import shutil
import os
import pandas as pd
import csv
import psycopg2
tsv_files = [i for i in os.listdir(os.getcwd()) if i[-3:] == 'tsv']

conn = psycopg2.connect("host=localhost dbname=imdb_final user=INFO7370 password=info7370")
cur = conn.cursor()
sql = """
DROP TABLE IF EXISTS public.{2};
CREATE TABLE public.{0}
({3}
)

WITH (
    OIDS = FALSE
)
TABLESPACE pg_default;

ALTER TABLE public.{1}
    OWNER to "INFO7370"
    """

copy =  """ COPY {0} FROM STDIN WITH CSV HEADER DELIMITER AS {1}
NULL AS {2} """
tab = "'\t'"
null = r"'\N'"

for i in tsv_files:
    with open('{}'.format(i), 'r', encoding="utf8") as file_r:
        filename = 'data/{}'.format(i)
        if not os.path.exists(os.path.dirname(filename)):
            os.makedirs(os.path.dirname(filename))
        
        with open('data/{}'.format(i), 'w', encoding="utf8")as file_w:
            for line in file_r:
                file_w.write(line.replace('"', '-'))
        
    with open('data/{}'.format(i), 'r', encoding="utf8") as file:
        reader = csv.reader(file, delimiter = '\t')
        for line in reader:
            cols = ', '.join(i+' '+str('VARCHAR NULL') for i in line)
            break
        print(i)
        table = i[:-4]
        cur.execute(query = sql.format(table, table, table, cols))
        conn.commit()
        cur.copy_expert(sql = copy.format(table, tab, null) , file = file)
        conn.commit()

akas.tsv
basics.tsv
crew.tsv
imdb.tsv
name.tsv
principals.tsv
ratings.tsv


In [140]:
conn.close()

In [ ]:
import pandas as pd
df_credit = pd.read_csv("tmdb_5000_credits.csv")
df_movies = pd.read_csv("tmdb_5000_movies.csv")

In [ ]:
import json
def json_to_series(text):
    keys, values = zip(*[item for dct in json.loads(text) for item in dct.items()])
    return pd.Series(values, index=keys)

In [ ]:
result_credit= pd.concat([df_credit, df_credit['cast'].apply(json_to_series ), df_credit['crew'].apply(json_to_series)], axis= 1) 

In [ ]:
dw = '''CREATE table aggregated_data AS /*+DIRECT*/ (
	
	
SELECT 	principals.tconst
-- 		ak.title,
-- 		ak.region,
-- 		ak.language, 
-- 		ak.types,
-- 		ak.attributes,
-- 		b.titletype,
-- 		b.isadult, 
-- 		b.runtimeminutes,
-- 		b.genres,
-- 		cr.directors,
-- 		r.averagerating,
-- 		r.numvotes,
-- 		i.knownfortitles,
-- 		i.primaryname
FROM principals
LEFT JOIN basics		ON (principals.tconst = basics.tconst)
--LEFT JOIN crew cr 		ON (cr.tconst = principals.tconst)
--LEFT JOIN imdb i 		ON (i.nconst = principals.nconst)
--LEFT JOIN akas ak 		ON (ak.titleid = principals.tconst)
--LEFT JOIN ratings r		ON (r.tconst = principals.tconst)'''

cur.execute(query = dw)
conn.commit()